In [1]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import faiss
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [ ]:
loader = PyPDFLoader(file_path='../datas.pdf')
doc = loader.load()

In [ ]:
doc

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
    chunk_overlap= 100, 
    separators=["\n\n","\n"," ",""]) 
chunks = text_splitter.split_documents(documents= doc)

In [ ]:
chunks[0:3]

In [2]:
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name='../LocalModel',
    model_kwargs=model_kwargs,
)

e:\Programs\Py\IhmSqr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
knowledge_base = faiss.FAISS.from_documents(chunks, embeddings)
knowledge_base.save_local("../vectors")
print("Embeddings successfully saved in vector Database and saved locally")

In [ ]:
llm = Ollama(model='llama3:latest')

In [3]:
loaded_vectors=faiss.FAISS.load_local("../vectors", embeddings, allow_dangerous_deserialization=True)

In [4]:
retriever = loaded_vectors.as_retriever()

In [ ]:
retriever.invoke('Quelle est la capitale du Bénin ?')

In [5]:
parser = StrOutputParser()

In [6]:
template = """
Vous es Seth Rensei, un assistant routier. Vpus devriez répondre à la question de la fin en utilisant le contexte suivant.
Si la réponse n'est pas possible à retirer dans le contexte, dites simplement que vous ne connaissez pas.
N'essayez pas d'inventer de réponse.

Context : {context}\n

Question : {question}\n

"""

prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

In [7]:
result = RunnableParallel(context = retriever,question = RunnablePassthrough())

In [ ]:
chain = result | prompt | llm | parser

In [ ]:
chain.invoke("Quelles sont les hôpitaux que je peux trouver au 2er arrondissement ?")

In [8]:
from langchain_g4f import G4FLLM
from g4f import Provider, models
from langchain.llms.base import LLM

In [9]:
gpt_llm: LLM = G4FLLM(
        model=models.gpt_35_turbo,
        provider=Provider.Aichatos,
    )

In [10]:
chain2 = result | prompt | gpt_llm | parser

In [11]:
chain2.invoke('Combien d\'arrondissement compte Cotonou ?')

New g4f version: 0.3.0.9 (current: 0.3.0.8) | pip install -U g4f


'Cotonou compte un total de treize arrondissements.'

In [ ]:
query = ''
docs = loaded_vectors.similary_search(query)
input_data = {
    'input_documents': docs,
    'question': query,
}